In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Attentionは効果的で重要なテクニック

# Attentionの仕組み
「注意機構」(Attention mechanism)は，seq2seqに必要な情報へ注意を向けさせる

## seq2seqの問題点
Encoderの出力するベクトルの長さが固定長だと，洋服がタンスからあふれだすように，いずれ限界が訪れる．

## Encoderの改良
LSTMレイヤの隠れ状態ベクトルを「すべて」利用することで，入力される文章の長さに応じた情報を出力させる  
全ての時刻の隠れ状態を返すか，最後の隠れ状態ベクトルだけを返すかは，KerasのRNN初期化時のreturn_sequencesなどで変更できる  
<br>
Encoderの出力するベクトルは入力された単語の数だけベクトルがあり，それぞれのベクトルは各単語に対応した情報を多く含む
<br>
吾輩 は 猫 で ある -> $hs$ 
<div style="text-align: center">a b c d (「吾輩」要素強め)<br>
e f g h<br>
i j k l<br>
m n o p<br>
q r s t<br>
</div>
<br>

ここまでのRNNは過去の情報しか見えないが，全体の情報をバランスよく含めたければ**双方向RNN**や**双方向LSTM**を使う．  
ここでは，これまで通り単方向のLSTMを利用する．

## Decoderの改良1
$hs$をDecoderで扱うとき，どの行がどの単語に対応しているかを学習させる．  
単語やフレーズの対応関係を表す情報は**アライメント**と呼ばれる．  
Attentionという技術は，アライメントのアイデアをseq2seqで自動で取り入れることに成功した．  